## Convert *existing* paradigm (PsychToolbox input) into FreeSurfer-friendly .par files

In [1]:
import os
import pandas as pd
import numpy as np

behav_dir='/autofs/space/lilli_001/users/DARPA-newARC/behavior/params'
par_path='/autofs/space/lilli_001/users/DARPA-newARC/behavior/params/optseq'

df = pd.read_csv(os.path.join(par_path,'mri_newarc.csv'),index_col=False)
df=pd.DataFrame(df)

display(df)

ConflictType=[]
ITI=[]
Reward=[]
Risk=[]
Trial=[]
Conflict=[]
## Replace Slope with log reg results
Slope=np.zeros(2*len(df))


for i in df.ConflictType:
    ConflictType.append('NULL     ')
    ConflictType.append(i)
    
for i in df.ITI:
    ITI.append(i) 
    ITI.append(5.00)

Time=[0]
s=0

for i in ITI:
    s=s+i
    Time.append(s)

Time=Time[:-1]

for ct in zip(ConflictType):
    if 'lowconflict' in ct:
        Conflict.append('1')
    elif 'medconflict' in ct:
        Conflict.append('2')
    elif 'highconflict' in ct:
        Conflict.append('3')
    elif 'NULL     ' in ct:
        Conflict.append('0')
    else: 
        print ct

df2_data={'RiskOnset':Time,'Conflict':Conflict,'Duration':ITI,'Slope':Slope,'ConflictType':ConflictType}
df2=pd.DataFrame(df2_data)

# print len(df2_data['Time'])
# print len(df2_data['Conflict'])
# print len(df2_data['ITI'])
# print len(df2_data['Slope'])
# print len(df2_data['ConflictType'])

trial_1 = pd.DataFrame(df2.iloc[0:162])
trial_2 = pd.DataFrame(df2.iloc[162:324])
trial_3 = pd.DataFrame(df2.iloc[324::])

trial_1.to_csv(os.path.join(par_path,'3cond_mod-001.csv'),index=False)
trial_2.to_csv(os.path.join(par_path,'3cond_mod-002.csv'),index=False)
trial_3.to_csv(os.path.join(par_path,'3cond_mod-003.csv'),index=False)
df2.to_csv(os.path.join(par_path,'3cond_mod-AllRuns.csv'),index=False)

,ConflictType,ITI,Reward,Risk,Trial
0,medconflict,1.750,7.317901,0.19,1
1,highconflict,1.750,13.901235,0.85,2
2,highconflict,2.625,9.756173,0.79,3
3,medconflict,2.625,5.367284,0.44,4
4,highconflict,1.750,7.317901,0.62,5
5,lowconflict,2.625,11.706790,0.45,6
6,lowconflict,1.750,11.462963,0.22,7
7,medconflict,5.250,14.754630,0.74,8
8,medconflict,2.625,13.291667,0.66,9
9,medconflict,2.625,5.854938,0.55,10


## Configure files to be FreeSurfer Compatible

In [2]:
import os
import pandas as pd
import numpy as np

bdir='/autofs/space/lilli_001/users/DARPA-newARC/behavior/behav_runs'
indiv_par_path='/autofs/space/lilli_001/users/DARPA-newARC/behavior/indiv_par_files'
par_path='/autofs/space/lilli_001/users/DARPA-newARC/behavior/params/optseq'

# subjects = ['newARC_001','newARC_004','newARC_005','newARC_008',\
#             'newARC_009','newARC_010','newARC_011','newARC_014',\
#             'newARC_015','newARC_016','newARC_017','newARC_018',\
#             'newARC_020']

subjects = ['newARC_001','newARC_004','newARC_005','newARC_008',\
            'newARC_009','newARC_010','newARC_011','newARC_014',\
            'newARC_015','newARC_016','newARC_017','newARC_018']

for subject in subjects:
    csv=os.path.join(bdir,'%s_run_1.csv' % subject)
    onsets=[]
    df=pd.DataFrame.from_csv(csv)
    for fix_on,rew_on in zip(df['FixOnset'],df['RiskOnset']):
        onsets.append(fix_on)
        onsets.append(rew_on)
    run1=os.path.join(par_path,'3cond_mod-001.csv')
    df1=pd.read_csv(run1)
    df1=pd.DataFrame(df1,columns=['Conflict','Duration','Slope','ConflictType','RiskOnset'])
    df1['Time'] = onsets
    display(df1)
        
#     trial_1.to_csv(os.path.join(indiv_par_path,'%s_run1.par'% subject),sep='\t',index=False,header=None)

,Conflict,Duration,Slope,ConflictType,RiskOnset,Time
0,0,1.750,0.0,NULL,0.000,0.024643
1,2,5.000,0.0,medconflict,1.750,1.777002
2,0,1.750,0.0,NULL,6.750,6.777979
3,3,5.000,0.0,highconflict,8.500,8.547789
4,0,2.625,0.0,NULL,13.500,13.531360
5,3,5.000,0.0,highconflict,16.125,16.167128
6,0,2.625,0.0,NULL,21.125,21.168377
7,2,5.000,0.0,medconflict,23.750,23.795017
8,0,1.750,0.0,NULL,28.750,28.792880
9,3,5.000,0.0,highconflict,30.500,30.543452


,Conflict,Duration,Slope,ConflictType,RiskOnset,Time
0,0,1.750,0.0,NULL,0.000,0.026733
1,2,5.000,0.0,medconflict,1.750,1.785311
2,0,1.750,0.0,NULL,6.750,6.786393
3,3,5.000,0.0,highconflict,8.500,8.555162
4,0,2.625,0.0,NULL,13.500,13.539514
5,3,5.000,0.0,highconflict,16.125,16.159374
6,0,2.625,0.0,NULL,21.125,21.159870
7,2,5.000,0.0,medconflict,23.750,23.812316
8,0,1.750,0.0,NULL,28.750,28.796871
9,3,5.000,0.0,highconflict,30.500,30.570680


,Conflict,Duration,Slope,ConflictType,RiskOnset,Time
0,0,1.750,0.0,NULL,0.000,0.124341
1,2,5.000,0.0,medconflict,1.750,1.856485
2,0,1.750,0.0,NULL,6.750,6.790634
3,3,5.000,0.0,highconflict,8.500,8.549954
4,0,2.625,0.0,NULL,13.500,13.546724
5,3,5.000,0.0,highconflict,16.125,16.181616
6,0,2.625,0.0,NULL,21.125,21.164352
7,2,5.000,0.0,medconflict,23.750,23.800127
8,0,1.750,0.0,NULL,28.750,28.801437
9,3,5.000,0.0,highconflict,30.500,30.554489


,Conflict,Duration,Slope,ConflictType,RiskOnset,Time
0,0,1.750,0.0,NULL,0.000,0.014883
1,2,5.000,0.0,medconflict,1.750,1.783063
2,0,1.750,0.0,NULL,6.750,6.784203
3,3,5.000,0.0,highconflict,8.500,8.530830
4,0,2.625,0.0,NULL,13.500,13.528559
5,3,5.000,0.0,highconflict,16.125,16.162472
6,0,2.625,0.0,NULL,21.125,21.160230
7,2,5.000,0.0,medconflict,23.750,23.794132
8,0,1.750,0.0,NULL,28.750,28.795182
9,3,5.000,0.0,highconflict,30.500,30.559318


,Conflict,Duration,Slope,ConflictType,RiskOnset,Time
0,0,1.750,0.0,NULL,0.000,0.019407
1,2,5.000,0.0,medconflict,1.750,1.788079
2,0,1.750,0.0,NULL,6.750,6.767904
3,3,5.000,0.0,highconflict,8.500,8.537014
4,0,2.625,0.0,NULL,13.500,13.532878
5,3,5.000,0.0,highconflict,16.125,16.156314
6,0,2.625,0.0,NULL,21.125,21.157593
7,2,5.000,0.0,medconflict,23.750,23.781733
8,0,1.750,0.0,NULL,28.750,28.796063
9,3,5.000,0.0,highconflict,30.500,30.563500


...

,Conflict,Duration,Slope,ConflictType,RiskOnset,Time
0,0,1.750,0.0,NULL,0.000,0.018512
1,2,5.000,0.0,medconflict,1.750,1.787102
2,0,1.750,0.0,NULL,6.750,6.783638
3,3,5.000,0.0,highconflict,8.500,8.540253
4,0,2.625,0.0,NULL,13.500,13.532105
5,3,5.000,0.0,highconflict,16.125,16.166809
6,0,2.625,0.0,NULL,21.125,21.163689
7,2,5.000,0.0,medconflict,23.750,23.814546
8,0,1.750,0.0,NULL,28.750,28.799038
9,3,5.000,0.0,highconflict,30.500,30.551063


,Conflict,Duration,Slope,ConflictType,RiskOnset,Time
0,0,1.750,0.0,NULL,0.000,0.162354
1,2,5.000,0.0,medconflict,1.750,1.855066
2,0,1.750,0.0,NULL,6.750,6.786086
3,3,5.000,0.0,highconflict,8.500,8.536940
4,0,2.625,0.0,NULL,13.500,13.540174
5,3,5.000,0.0,highconflict,16.125,16.194281
6,0,2.625,0.0,NULL,21.125,21.166379
7,2,5.000,0.0,medconflict,23.750,23.819489
8,0,1.750,0.0,NULL,28.750,28.807256
9,3,5.000,0.0,highconflict,30.500,30.577053


,Conflict,Duration,Slope,ConflictType,RiskOnset,Time
0,0,1.750,0.0,NULL,0.000,0.027443
1,2,5.000,0.0,medconflict,1.750,1.778111
2,0,1.750,0.0,NULL,6.750,6.798032
3,3,5.000,0.0,highconflict,8.500,8.549452
4,0,2.625,0.0,NULL,13.500,13.548080
5,3,5.000,0.0,highconflict,16.125,16.174419
6,0,2.625,0.0,NULL,21.125,21.173013
7,2,5.000,0.0,medconflict,23.750,23.798713
8,0,1.750,0.0,NULL,28.750,28.797791
9,3,5.000,0.0,highconflict,30.500,30.548305


,Conflict,Duration,Slope,ConflictType,RiskOnset,Time
0,0,1.750,0.0,NULL,0.000,0.156469
1,2,5.000,0.0,medconflict,1.750,1.841484
2,0,1.750,0.0,NULL,6.750,6.784002
3,3,5.000,0.0,highconflict,8.500,8.535473
4,0,2.625,0.0,NULL,13.500,13.534528
5,3,5.000,0.0,highconflict,16.125,16.173028
6,0,2.625,0.0,NULL,21.125,21.180915
7,2,5.000,0.0,medconflict,23.750,23.798954
8,0,1.750,0.0,NULL,28.750,28.801904
9,3,5.000,0.0,highconflict,30.500,30.555583


,Conflict,Duration,Slope,ConflictType,RiskOnset,Time
0,0,1.750,0.0,NULL,0.000,0.156472
1,2,5.000,0.0,medconflict,1.750,1.852902
2,0,1.750,0.0,NULL,6.750,6.786120
3,3,5.000,0.0,highconflict,8.500,8.555893
4,0,2.625,0.0,NULL,13.500,13.543390
5,3,5.000,0.0,highconflict,16.125,16.180919
6,0,2.625,0.0,NULL,21.125,21.168554
7,2,5.000,0.0,medconflict,23.750,23.798704
8,0,1.750,0.0,NULL,28.750,28.795580
9,3,5.000,0.0,highconflict,30.500,30.563382


## Concatenate par files